### Testing the postcode.io api to retrieve lat lon per postcode

#### imports

In [2]:
import pandas as pd
import numpy as np
import requests

In [28]:
%cd ..

/Users/Marie/code/MarieMCodes/re_value_predictor


/Users/Marie/.pyenv/versions/3.10.6/envs/re_value_predictor/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [75]:
from project_code.data import load_csv, tidy_df

In [68]:
%load_ext autoreload
%autoreload 2

#### testing api

api.postcodes.io/postcodes

{
  "postcodes" : ["OX49 5NU", "M32 0JG", "NE30 1DP"]
}
bulk upload = 100 postcodes max

https://api.postcodes.io/postcodes/N79EE

RESULT:
{
"status": 200,
"result": {
    "postcode": "N7 9EE",
    "quality": 1,
    "eastings": 530242,
    "northings": 184992,
    "country": "England",
    "nhs_ha": "London",
    "longitude": -0.123025,
    "latitude": 51.548796,



In [93]:
def postcode_api(postcode):
    url=f"https://api.postcodes.io/postcodes/{postcode}"
    try:
        response=requests.get(url).json()
        #print (response)
        #print (response["result"])
        lon=response["result"]["longitude"]
        lat=response["result"]["latitude"]
        return lat, lon
    except:
        return "ERROR"

postcode=np.nan
postcode_api(postcode)

'ERROR'

In [ ]:
# trying with bulk postcodes but doesn't work
def postcodes_api(postcodes):
    url=f"https://api.postcodes.io/{postcodes}"
    try:
        response=requests.get(url).json()
        #print (response)
        #print (response["result"])
        lon=response["result"]["longitude"]
        lat=response["result"]["latitude"]
        return lat, lon
    except:
        return "ERROR"

postcodes={"postcodes":["N7 9EE","W1G 8NT"]}
postcodes_api(postcodes)

In [31]:
url=f"api.postcodes.io/postcodes/{postcode}"
url

"api.postcodes.io/postcodes/['N7 9EE']"

### Try it on sample data

In [55]:
#data=load_csv().sample(10000,random_state=42)
data=load_csv()

In [94]:
df=tidy_df(data)

In [95]:
len(df)

3566539

In [49]:
# testing on smaller df
df.columns
df=df.drop(columns=['price', 'date', 'property_type', 'property_age',
       'ownership', 'number', 'additional_info', 'locality', 'town', 'county', 'year', 'month', 'day'])

In [53]:
# picking smaller sample to call api, and test time (ca. 4 min)
cols=['borough','full_property_number','street','postcode']
df=df[cols]
df=df.sample(1000,random_state=42)

In [54]:
# calling api and filling df
df['lat_lon']=df['postcode'].apply(postcode_api)
df

,postcode,street,borough,full_property_number,lat_lon
1430780,WC1X 0LQ,CUBITT STREET,CAMDEN,2 FLAT 9,"(51.526891, -0.114642)"
198929,NW5 1SU,DARTMOUTH PARK ROAD,CAMDEN,21,"(51.558238, -0.145084)"
3441998,SW12 0PD,EMMANUEL ROAD,LAMBETH,HEATHWOOD COURT 10,"(51.441927, -0.142131)"
3110629,TW8 8FN,DURHAM WHARF DRIVE,HOUNSLOW,COPLAND COURT FLAT 5,"(51.484431, -0.313186)"
3476006,SM3 8NR,FIELDSEND ROAD,SUTTON,18,"(51.363632, -0.215272)"
...,...,...,...,...,...
3393216,E15 4JY,BOLTON ROAD,NEWHAM,28A,"(51.542532, 0.016057)"
1781599,SE15 5NR,DENMAN ROAD,SOUTHWARK,68,"(51.471882, -0.075816)"
1250877,N12 7HG,SOUTHOVER,BARNET,85,"(51.622869, -0.192251)"
2790328,SE26 5NY,BAILEY PLACE,BROMLEY,KITE HOUSE FLAT 6,"(51.420012, -0.051847)"


#### Call api on full dataset

In [90]:
# drop postcode nans - kept even the nan postcodes, might be able to fill those with the openstreet api
# df=df.dropna(subset=['postcode'])

In [ ]:
# call the api and fill whole df
df['lat_lon']=df['postcode'].apply(postcode_api)
df

In [ ]:
# split into 2 columns lat and lon
df['lat']=df['lat_lon'].apply(lambda x: x[0])
df['lon']=df['lat_lon'].apply(lambda x: x[1])


In [ ]:
# write to csv
